In [1]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
data_dirs = ['../data/results_D100', '../data/results_D25']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [3]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[1].split('/')[0]
    experiment = fname_pieces[2]
    n_mca = int(fname_pieces[3])
    target = fname_pieces[-3]
    classif = fname_pieces[-2]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Target": target,
        "Experiment": experiment,
        "MCA Simulations": n_mca,
        "Classifier": classif,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and n_mca != 20:
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [4]:
df_results

,Target,Experiment,MCA Simulations,Classifier,Dataset,rs,aggregation,acc,f1,test_acc,test_f1,test_mean_acc,test_mean_f1,n_models
0,vo2max,mca,10,LR,D100,42,mega,0.482848,0.527828,0.563636,0.578947,0.517273,0.501799,5
1,vo2max,mca,2,LR,D100,42,median,0.615833,0.646527,0.550000,0.526316,0.490000,0.477980,5
2,sex,mca,20,RF,D100,42,consensus,0.653333,0.752597,0.550000,0.571429,0.540000,0.588594,5
3,age,mca,20,SVM,D100,42,ref,0.731667,0.803382,0.550000,0.709677,0.570000,0.691077,5
4,bmi,mca,10,SVM,D100,42,none,0.589767,0.465334,0.631000,0.650274,0.736200,0.700024,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,age,mca,2,LR,D25,42,median,0.550000,0.366667,0.600000,0.500000,0.480000,0.460000,5
953,age,subsample,20,SVM,D25,42,mega,0.550000,0.366667,0.400000,0.400000,0.400000,0.400000,5
954,age,session,20,LR,D25,42,ref,0.550000,0.426667,0.400000,0.400000,0.280000,0.360000,5
955,vo2max,session,20,LR,D25,42,none,0.670000,0.742762,0.540000,0.632857,0.548000,0.620190,50


In [5]:
df_results.to_csv("../data/aggregation_results.csv", index=False)